# Build and Deploy an in-Db2 Machine Learning Model with Db2 v11.5

In this workshop we will use Db2's native machine learning algorithms to build a customer segmentation model using the K-Means clustering algorithm. Our sample dataset tells us about customers at a bank and contains information such as their age, employment status, marital status, education level, and bank balance. We will use this data and the K-Means algorithm to segment these customers, allowing the buisness to more effectively target specific groups in their customer base.

**Prerequisites**: If you are unfamiliar with the Jupyter Notebook environment, it is recommended to have gone through the [An Introduction to Jupyter Notebooks](./An_Introduction_to_Jupyter_Notebooks.ipynb) lab first

## Imports and Connection to Db2

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

First load the Db2 magic functions and make a connection to our Db2 database.

In [ ]:
%run db2.ipynb

In [ ]:
%run connectiondb2ml-banking.ipynb

In [ ]:
%sql option maxrows -1

## Read in Data

We will first make a copy of our training and test data to a new schema `CLUSTERING`. This will allow us to modify the data without affecting the underlying raw data

In [ ]:
%%sql
CREATE TABLE CLUSTERING.TRAIN AS (SELECT * FROM BANK.TRAIN) WITH DATA

In [ ]:
%%sql
CREATE TABLE CLUSTERING.TEST AS (SELECT * FROM BANK.TEST) WITH DATA

Let's take a look at the first 10 rows of our training data

In [ ]:
%%sql
SELECT * FROM CLUSTERING.TRAIN ORDER BY ID FETCH FIRST 10 ROWS ONLY 

## Data Exploration

Before we begin to modify our data, we must first understand the underlying distributions. We will look at the number of missing values, cardinality (number of unique values), and variance (this tells us what range of values the feature takes). We will use the `COLUMN_PROPERTIES` stored procedure to collect summary statistics about the features in our dataset.

Input Parameters:
- intable: Name of the input table
- outtable: The output table that contains the column properties data
- incolumn: The columns of the input table that have specific properties
- withstatistics: A flag that indicates whether statistical properties of the columns should be collected

[IDAX.COLUMN_PROPERTIES Documentation](https://www.ibm.com/docs/en/db2/11.5?topic=exploration-idaxcolumn-properties-create-column-properties-table)

In [ ]:
%%sql
CALL IDAX.COLUMN_PROPERTIES('intable=CLUSTERING.TRAIN,outtable=CLUSTERING.COLPROP,incolumn=ID:id,
                            withstatistics=true')

In [ ]:
%%sql
SELECT * FROM CLUSTERING.COLPROP ORDER BY COLNO

### Missing Values

Let's now look at the number of missing values using the column properties table we just created. We can use the query below to inquire about the number of missing values and the total percentage of missing values in each feature.

In [ ]:
%%sql
SELECT COLNO, NAME, TYPE, NUMMISSING, NUMMISSING + NUMINVALID + NUMVALID as NUMBER_OF_VALUES, 
ROUND(DEC(NUMMISSING,10,2)/(DEC(NUMMISSING, 10,2)+DEC(NUMINVALID, 10,2)+DEC(NUMVALID, 10,2))*100,2) 
AS PERCENT_NULL
FROM CLUSTERING.COLPROP WHERE NUMMISSING > 0 ORDER BY PERCENT_NULL DESC

We can see that the `AGE` feature has some missing values that require imputation.

#### Hands-On Exercise 1

Most machine learning algorithms prefer for the categorical features (i.e., columns) in the dataset to not contain too many unique values. Additionally, they prefer for numerical features and to be similarly scaled. Therefore we want to ensure that both the cardinality of our categorical features (i.e., number of unique values) and the maximum range of our numerical features remain low.

1. **Using the column properties table we have just created identify any categorical columns with high cardinality**
2. **Using the column properties table identify any continuous columns with the widest ranges of values**

In [ ]:
### YOUR CODE HERE ###

## Data Transformation

We are now ready to perform some data transformations on our training data. The transformations will be missing value imputation and standardization

### Missing Value Imputation

Most machine learning algorithms cannot work with missing values. We first use the `IMPUTE_DATA` stored procedure to impute (i.e., replace) missing values in the `AGE` feature in the training data using the mean value of the column.

Input Parameters:
- intable: Name of the input table
- method: The method of data imputation.
- incolumn: The columns of the input table where the missing values must be replaced

[IDAX.IMPUTE_DATA Documentation](https://www.ibm.com/docs/en/db2/11.5?topic=transformation-idaximpute-data-impute-missing-data)

In [ ]:
%%sql
CALL IDAX.IMPUTE_DATA('intable=CLUSTERING.TRAIN, method=mean,incolumn=AGE')

Next, we perform the same imputation on the testing data, ensuring that we are imputing the missing values with the mean value in the *training* data.

In [ ]:
%%sql
UPDATE CLUSTERING.TEST
SET AGE = (SELECT MEAN FROM CLUSTERING.COLPROP WHERE NAME = 'AGE')
WHERE AGE IS NULL

### Standardization

Most machine learning algorithms do not perform well when the numerical attributes in the data have very different scales (i.e., large range of possible values). We will use the `STD_NORM` stored procedure to standardize the feature `BALANCE` as it takes on a large range of values. A standardized value S is equal to (x-mean)/stddev, where x is the input value, mean is the mean value of the column, and stddev is the standard deviation of the column. The standardized values range from minus infinity to plus infinity.

Input Parameters:
- intable: Name of the input table
- id: The column of the input table that identifies a unique instance ID
- incolumn: The columns of the input table that are to be considered. Flags: L=leave unchanged, S=Standardize
- outtable: The output table that contains the modified data

[IDAX.STD_NORM Documentation](https://www.ibm.com/docs/en/db2/11.5?topic=transformation-idaxstd-norm-standardize-normalize-columns-input-table)


In [ ]:
%%sql
CALL IDAX.STD_NORM('intable=CLUSTERING.TRAIN, id=ID, outtable=CLUSTERING.TRAIN_CLEAN,
                   incolumn=AGE:L;JOB:L;MARITAL:L;EDUCATION:L;DEFAULT:L;BALANCE:S;
                   HOUSING:L;LOAN:L;CONTACT:L;DAY:L;MONTH:L;DURATION:L;CAMPAIGN:L;PDAYS:L;
                   PREVIOUS:L;POUTCOME:L;SUBSCRIBED:L')

Next, we perform the same standardization on the testing data, ensuring that we are using the mean and standard deviation in the *training* data. We will also rename the column so that it is consistent with the new column name in the training data.

In [ ]:
%%sql
ALTER TABLE CLUSTERING.TEST ADD STD_BALANCE DOUBLE;

UPDATE CLUSTERING.TEST
SET STD_BALANCE = (CAST(BALANCE AS FLOAT)-(SELECT MEAN FROM CLUSTERING.COLPROP WHERE NAME ='BALANCE'))/
(SELECT SQRT(VARIANCE) FROM CLUSTERING.COLPROP WHERE NAME ='BALANCE');

CREATE TABLE CLUSTERING.TEST_CLEAN AS (SELECT ID,AGE,JOB,MARITAL,EDUCATION,DEFAULT,STD_BALANCE,HOUSING,LOAN,
                                       CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,SUBSCRIBED 
                                       FROM CLUSTERING.TEST) WITH DATA

## Model Training

Now that we have cleaned versions of our training and test data, we are ready to begin model training. We will use a K-means clustering model. The objective of K-means is simple: group similar data points together and discover underlying patterns.The K-means algorithm identifies k number of centroids (i.e., cluster centers), and then allocates every data point to the nearest cluster, while keeping the centroids as small as possible.


![Kmeans](media/kmeans.png)

**Figure 1:** This figure depicts an example of an idea clustering scenario. The different coloured points represent different clusters, while the large black dots represent the cluster centers (centroids).

We will train our model using the `KMEANS` stored procedure with default parameter values and k=3.

Input Parameters:
- model: The name of the clustering model that is to be built
- intable: The name of the input table
- id: The column of the input table that identifies a unique instance ID
- outtable: The name of the output table where the clusters are assigned to each input table record
- randseed: The random seed for the generator
- k: The number of clusters to build


[IDAX.KMEANS Documentation](https://www.ibm.com/docs/en/db2/11.5?topic=functions-idaxkmeans-build-k-means-clustering-model)

In [ ]:
%%sql
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_3,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_3_OUT,
                 randseed=42,k=3')

We can query the model tables to get more information about the model we have just trained

In [ ]:
%%sql
SELECT * FROM CLUSTERING.KMEANS_3_MODEL;

The cluster table tells us about the clusters the model has created. Some features of note:
- **SIZE**: Number of data records in the cluster
- **RELSIZE**: Relative size of the cluster, that is, SIZE/Total number of records
- **WITHINSS**: A measure of the cluster homogeneity, that is, the sum of squared distances between records of the cluster and the cluster center.

In [ ]:
%%sql
SELECT * FROM CLUSTERING.KMEANS_3_CLUSTERS ORDER BY CLUSTERID;

### Model Tuning

We now need to determine the optimal value for k (number of clusters) in our model. A value of k that is too small will underfit our training data and will not be able to make useful predictions. A value of k that is too large will overfit our training data and will not be able to generalize well to new data. We will train models for k=3 to k=11 and then plot the mean sum of squared distances vs. k to determine the optimal value of k

In [ ]:
%%time
%%sql
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_4,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_4_OUT,
randseed=42,k=4,distance=euclidean');
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_5,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_5_OUT,
randseed=42,k=5,distance=euclidean');
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_6,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_6_OUT,
randseed=42,k=6,distance=euclidean');
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_7,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_7_OUT,
randseed=42,k=7,distance=euclidean');
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_8,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_8_OUT,
randseed=42,k=8,distance=euclidean');
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_9,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_9_OUT,
randseed=42,k=9,distance=euclidean');
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_10,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_10_OUT,
randseed=42,k=10,distance=euclidean');
CALL IDAX.KMEANS('model=CLUSTERING.KMEANS_11,intable=CLUSTERING.TRAIN_CLEAN,id=ID,outtable=CLUSTERING.KMEANS_11_OUT,
randseed=42,k=11,distance=euclidean');

In [ ]:
# Collect the mean sum of squared distances for each model k=3 to k=11
ss_list = []

for k in range(3,12):
    table_name = "CLUSTERING.KMEANS_"+str(k)+"_CLUSTERS"
    # Select the mean sum of squared distances and append to a list for later plotting
    query = %sql SELECT AVG(WITHINSS) as MEAN_SS FROM {table_name}
    df = pd.DataFrame(query)
    value = df.iloc[0]['MEAN_SS']
    ss_list.append(value)

In [ ]:
# Plot mean sum of squared distances vs. k
k=range(3,12)
plt.plot(k,ss_list);
plt.xlabel('k')
plt.ylabel('Mean sum of sqaured distances')
plt.title('Elbow method of determining optimal k value');

To select the optimal k value, we look at the "elbow point" in the graph - in our case we will select k=6.

## Make Predictions

With our final model selected, we can now apply this model to the test data using the `PREDICT_KMEANS` stored procedure and assign each record to a cluster.

Input Parameters:
- model: The name of the clustering model
- intable: The name of the input table
- id: The column of the input table that identifies a unique instance ID
- outtable: he name of the output table where the assigned clusters are stored


[IDAX.PREDICT_KMEANS Documentation](https://www.ibm.com/docs/en/db2/11.5?topic=functions-idaxpredict-kmeans-apply-k-means-clustering-model)

In [ ]:
%%sql
CALL IDAX.PREDICT_KMEANS('model=CLUSTERING.KMEANS_6,intable=CLUSTERING.TEST_CLEAN,
                         outtable=CLUSTERING.PREDICTIONS,id=ID')

We can query the `CLUSTERING.PREDICTIONS` table to look at our model's predictions.

The table contains the following features:
- ID: The unique row identifier
- CLUSTER_ID: The cluster the row is mapped to
- DISTANCE: The distance between the input data point and the center of its assigned cluster.

In [ ]:
%%sql
SELECT * FROM CLUSTERING.PREDICTIONS ORDER BY ID FETCH FIRST 10 ROWS ONLY 

#### Hands-On Exercise 2

Return to the output of the `CLUSTERS` model table for the k=3 model (cell 17). Note that one of the clusters contains a large proportion of all the data points.

**Using the CLUSTERS model tables for our final model (k=6), print the summary of the clusters. What does this say about our choice of k?**

In [ ]:
### YOUR CODE HERE ###

## Analyze the Results

We can query the `KMEANS_6_OUT` table to find our archetypal customer for each cluster. The archetypal customer is the customer most representative of their group, and is therefore closest to the center of their respective cluster.

In [ ]:
%%sql 
SELECT * FROM CLUSTERING.KMEANS_6_OUT

In [ ]:
# Find the customers in the training set who are closest to their respective clusters
query = %sql SELECT * FROM CLUSTERING.KMEANS_6_OUT
df = pd.DataFrame(query)
df_arch = df.groupby(['CLUSTER_ID']).min()

# Add their ID's to a list - for use in following query
arch_ids = ', '.join([str(val) for val in df_arch['ID'].values])
df_arch.head(6)

We can now use this list of IDs to map back to our training data and get more information about these customers.

In [ ]:
%sql SELECT * FROM CLUSTERING.TRAIN_CLEAN WHERE ID IN ({arch_ids})

#### Hands-On Exercise 3

Suppose we chose a larger k value for our model, say k = 11.

**Who are the archetypal customers for a k=11 model? Do they differ from those in the k=6 model?**

In [ ]:
### YOUR CODE HERE ###

## Visualize the Results

Since the data has 17 features, it would not be feasible to show how they compare in a 3-D plot. We can use a technique called Principal Component Analysis (PCA) to project the data into a lower dimension (17 -> 3). The key is perserving variance after transforming the data. 

PCA is an algorithm that generates "principal components" that are used to project the data onto. Each component is ranked by how much variance exsists in the data after projection. We can then choose the best n components to use to represent the data.

We do this here using the `IDAX.PCA` stored procedure.

Input Parameters:
- model: The name of the PCA model that is to be built
- intable: The name of the input table
- id: The column of the input table that identifies a unique instance ID
- incolumn: The columns to be considered

[IDAX.PCA Documentation](https://www.ibm.com/docs/en/db2-warehouse?topic=functions-idaxpca-build-pca-model)

In [ ]:
%%sql 
CALL IDAX.PCA('model=CLUSTERING.PCA, intable=CLUSTERING.TRAIN_CLEAN, id=ID, 
              incolumn=AGE; JOB; MARITAL; EDUCATION; DEFAULT; STD_BALANCE; HOUSING; LOAN; 
              CONTACT; DAY; MONTH; DURATION; CAMPAIGN; PDAYS; PREVIOUS; POUTCOME; SUBSCRIBED');

Once a PCA representation is created, we project the data into 3 dimensions using `IDAX.PCA_PROJECT` with the parameter `pcnumber=3`.

Input Parameters:
- model: The name of the PCA model
- intable: The name of the input table
- outtable: The name of the output table where the predictions are stored.
- pcnumber: The number of principal components that are to be used for the projection.

[IDAX.PROJECT_PCA Documentation](https://www.ibm.com/docs/en/db2-warehouse?topic=functions-idaxproject-pca-apply-pca-model)

In [ ]:
%%sql 
CALL IDAX.PROJECT_PCA('model=CLUSTERING.PCA, 
                      intable=CLUSTERING.TEST_CLEAN, 
                      outtable=CLUSTERING.PCA_RESULTS_3,
                      pcnumber=3');

In [ ]:
%%sql
SELECT * FROM CLUSTERING.PCA_RESULTS_3 FETCH FIRST 10 ROWS ONLY

In [ ]:
# Join the PCA-transformed dataset with the K-Means model predictions
query = %sql SELECT a.*, b.CLUSTER_ID FROM CLUSTERING.PCA_RESULTS_3 as a INNER JOIN CLUSTERING.PREDICTIONS as b ON a.ID=b.ID ORDER BY ID
results = pd.DataFrame(query)

In [ ]:
results.head()

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(8, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot predictions vs principal component
scatter = ax.scatter(results["PC1"], results["PC2"], results["PC3"],  
                     c = results["CLUSTER_ID"], marker='o', alpha = 0.57)

# Add legend and titles
legend1 = ax.legend(*scatter.legend_elements(num=6),
                    loc="upper left", title="Cluster")
ax.add_artist(legend1)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3');
ax.set_title('Clustering Model Predictions vs. Principal Component');

## Clean Up

With our analysis done, we are ready to clean up our working schema. We will use the `IDAX.DROP_MODEL` stored procedure to delete the models and their associated model tables. Then we can use normal `DROP TABLE` statements to delete the rest of the tables in our schema.

In [ ]:
%%sql
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_3');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_4');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_5');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_6');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_7');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_8');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_9');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_10');
CALL IDAX.DROP_MODEL('model=CLUSTERING.KMEANS_11');
CALL IDAX.DROP_MODEL('model=CLUSTERING.PCA');

In [ ]:
# Drop all remaining tables in schema CLUSTERING
query = %sql select 'drop table '||rtrim(tabschema)||'.'||rtrim(tabname) from syscat.tables where tabschema = 'CLUSTERING'
tabs_to_drop = pd.DataFrame(query)
for cmd in tabs_to_drop['1']:
    %sql {cmd}